# Trace Collection: Different devices, different executables

---
We will be collecting traces from different devices. Each device will have mutliple traces fro different executables, such as ANSII+masked, MBEDTLS, HWAES

---

In [ ]:
SCOPETYPE = 'OPENADC'
#PLATFORM = 'CW308_STM32L4' #'CWLITEARM'
#PLATFORM = 'CW308_STM32F0'
PLATFORM = 'CW308_STM32F3'

# #CRYPTO_TARGET = "HWAES" #"MBEDTLS" #"TINYAES128C"
CRYPTO_TARGET = "MBEDTLS"
SS_VER = 'SS_VER_1_1' ##'SS_VER_2_1' or 'SS_VER_1_1'
# #CHECK_CORR = False ##From CW_lite code

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
scope.default_setup()
target = cw.target(scope)
# target.baud = 20834
print(target.baud)
print(scope)

In [ ]:
%%bash
cd ../../../hardware/victims/firmware/
mkdir -p simpleserial-aes-captureTrace_f303_081924_cwlite && cp -r simpleserial-aes/* $_

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-aes-captureTrace_f303_081924_cwlite
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3

In [ ]:
fw_path = '../../../hardware/victims/firmware/simpleserial-aes-captureTrace_f303_081924_cwlite/simpleserial-aes-{}.hex'.format(PLATFORM)
cw.program_target(scope, prog, fw_path)

In [ ]:
fw_path  = "/home/manoj/Documents/chipwhisperer-firmware/simpleserial-aes/simpleserial-aes_CW308-STM32F3_MBEDTLS.hex"
cw.program_target(scope, prog, fw_path)

In [ ]:
scope.default_setup()

In [ ]:
print(scope.adc.presamples)
print(scope.adc.timeout)
print(scope.clock.adc_src)
print(scope.clock.adc_freq)
print(scope.clock.adc_rate)
print(scope.clock.adc_locked)
print(scope.clock.freq_ctr)
print(scope.clock.freq_ctr_src)
print(scope.clock.clkgen_src)
print(scope.clock.extclk_freq)
print(scope.clock.clkgen_mul)
print(scope.clock.clkgen_div)
print(scope.clock.clkgen_freq)
print(scope.gain)

In [ ]:
#scope.adc.samples = 1500
#scope.adc.offset = 24400
#scope.clock.clkgen_freq = 4e6
scope.clock.adc_src = "clkgen_x1"
# target = cw.target(scope)
#target.baud = 20834
#scope.gain.db = 30
#scope.gain.gain = 40
print(target.baud)
#scope.adc.timeout = 60
#scope.adc.default()

In [ ]:
reset_target(scope)

In [ ]:
print(scope.adc.presamples)
print(scope.adc.samples)
print(scope.adc.timeout)
print(scope.clock.adc_src)
print(scope.clock.adc_freq)
print(scope.clock.adc_rate)
print(scope.clock.adc_locked)
print(scope.clock.freq_ctr)
print(scope.clock.freq_ctr_src)
print(scope.clock.clkgen_src)
print(scope.clock.extclk_freq)
print(scope.clock.clkgen_mul)
print(scope.clock.clkgen_div)
print(scope.clock.clkgen_freq)
print(scope.gain)

In [ ]:
##For 1500 CW downsamples, trace_key_1500
#scope.adc.decimate = 4
scope.adc.samples = 1500 ##Number of samples to take
scope.adc.presamples = 0

In [ ]:
#Capture Traces
from tqdm import tnrange, trange
import numpy as np
import time

ktp = cw.ktp.Basic()
ktp.fixed_key = False

traces = []
N = 2000  # Number of traces
prjDir = "/mnt/hgfs/f415_snr_data/"

project = cw.create_project(prjDir + "aes_powerTrace_capsRemoved_ldo_ext_dc_ir_2p33volt_trace.cwp", overwrite=True)

for i in trange(N, desc='Capturing traces:'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
#    key[0] = fileNum

    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)

try:
    print(scope.adc.trig_count) # print if this exists
except:
    pass
project.save()
project.close()

In [ ]:
np.random.choice(256,1)[0]

## NOP trace collection

In [ ]:
#Capture Traces
from tqdm import tnrange, trange
import numpy as np
import time

ktp = cw.ktp.Basic()
ktp.fixed_key = False

traces = []
N = 10  # Number of traces
project = cw.create_project("traces/f303_trial_081924_preCompiled.cwp", overwrite=True)

for i in trange(N, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
#    key[0] = 0
#     for k in range(len(key)):
#         key[k] = 150
        
    #print(f"key= {key}")

    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)

try:
    print(scope.adc.trig_count) # print if this exists
except:
    pass
project.save()

## Plot traces

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import chipwhisperer as cw

project = cw.open_project("traces/f051_flto.cwp")
print(project.location)
xShape = project.traces[0].wave.shape[0]
x=np.linspace(0,xShape-1, xShape)

#savePath = "/home/manojgopale/Documents/chipwhisperer/jupyter/courses/sca201/img/sca201_images/" + prjName + ".jpg"

fig, axs = plt.subplots(4, sharex=True, sharey= True, figsize=(30,10))
axs[0].plot(x,project.traces[0].wave)
axs[0].set_title(project.traces[0].key)
axs[1].plot(x,project.traces[1].wave)
axs[1].set_title(project.traces[1].key)
axs[2].plot(x,project.traces[8].wave)
axs[2].set_title(project.traces[8].key)
axs[3].plot(x,project.traces[9].wave)
axs[3].set_title(project.traces[9].key)

fig.savefig("img/f415_mbedtls_1500_nop_ldo_j1_cw_trial.png")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import chipwhisperer as cw
import plotly.graph_objects as go
from plotly.subplots import make_subplots

project = cw.open_project("traces/f303_trial_081924_preCompiled.cwp")

print(project.location)
xShape = project.traces[0].wave.shape[0]
x=np.linspace(0,xShape-1, xShape)
colorList = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00"]

fig = go.Figure()
fig = make_subplots(rows=1, cols=1, shared_yaxes=True, shared_xaxes=True, \
)
for idx in range(4):
    fig.add_trace(go.Scatter(x=x, y=project.traces[idx].wave, mode='lines', line=dict(color=colorList[idx]), name="key_" + str(project.traces[idx].key[0])))

fig.update_layout(
    showlegend=True,
    title="F303 + Pre Compiled Github + Gain.gain = 30"
)

savePath = "img/f303_trial_081924_preCompiled.html"
fig.write_html(savePath, auto_open=True)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import chipwhisperer as cw
import plotly.graph_objects as go
from plotly.subplots import make_subplots

project1 = cw.open_project("traces/f415_mbedtls_1500_ldo_extDC_disable_ir_0_turn_clockwise_potentiometer.cwp")
project2 = cw.open_project("traces/f415_mbedtls_1500_ldo_extDC_disable_ir_half_turn_clockwise_potentiometer.cwp")
project3 = cw.open_project("traces/f415_mbedtls_1500_ldo_extDC_disable_ir_full_turn_clockwise_potentiometer.cwp")
project4 = cw.open_project("traces/f415_mbedtls_1500_ldo_ext_dc.cwp")

print(project1.location)
xShape = project1.traces[0].wave.shape[0]
x=np.linspace(0,xShape-1, xShape)
colorList = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00"]

fig = go.Figure()
fig = make_subplots(rows=1, cols=1, shared_yaxes=True, shared_xaxes=True, \
)
fig.add_trace(go.Scatter(x=x, y=project4.traces[idx].wave, mode='lines', line=dict(color="black"), name="internal regulator ON"))
for idx in range(3):
    fig.add_trace(go.Scatter(x=x, y=project1.traces[idx].wave, mode='lines', line=dict(color=colorList[idx]), name="0turn" + str(idx)))
    fig.add_trace(go.Scatter(x=x, y=project2.traces[idx].wave, mode='lines', line=dict(color=colorList[idx+1]), name="half_" + str(idx)))
    fig.add_trace(go.Scatter(x=x, y=project3.traces[idx].wave, mode='lines', line=dict(color=colorList[idx+2]), name="full_" + str(idx)))

fig.update_layout(
    showlegend=True,
    title="AES Power trace + LDO Ext DC"
)

savePath = "img/f415_AES_trace_compare_ldo_ext_dc.html"
fig.write_html(savePath, auto_open=True)


In [ ]:
import numpy as np
import chipwhisperer as cw
import plotly.graph_objects as go
from plotly.subplots import make_subplots

dict_aes_ext = {
"1": "/mnt/hgfs/f415_snr_data/aes_powerTrace_ldo_ext_dc_trace.cwp",
"2": "/mnt/hgfs/f415_snr_data/aes_powerTrace_ldo_ext_dc_ir_1p295volt_trace.cwp",
"3": "/mnt/hgfs/f415_snr_data/aes_powerTrace_ldo_ext_dc_ir_2p316volt_trace.cwp",
"4": "/mnt/hgfs/f415_snr_data/aes_powerTrace_capsRemoved_ldo_ext_dc_ir_1p261volt_trace.cwp",
"5": "/mnt/hgfs/f415_snr_data/aes_powerTrace_capsRemoved_ldo_ext_dc_ir_1p29volt_trace.cwp",
"6": "/mnt/hgfs/f415_snr_data/aes_powerTrace_capsRemoved_ldo_ext_dc_ir_2p309volt_trace.cwp",
"7": "/mnt/hgfs/f415_snr_data/aes_powerTrace_capsRemoved_ldo_ext_dc_ir_2p33volt_trace.cwp",
}

project1 = cw.open_project(dict_aes_ext["1"])
xShape = project1.traces[0].wave.shape[0]
x=np.linspace(0,xShape-1, xShape)

colorList = ["#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00", "#e41a1c", "#377eb8"]

for idx, key in enumerate(dict_aes_ext.keys()):
    project = cw.open_project(dict_aes_ext[key])
    fig.add_trace(go.Scatter(x=x, y=project.traces[0].wave, mode='lines', line=dict(color=colorList[idx]), name="trace_" + str(idx)))
    
fig.update_layout(
    showlegend=True,
    title="AES Power trace + LDO Ext DC"
)

savePath = "/mnt/hgfs/f415_snr_data/f415_AES_trace_compare_ldo_ext_dc.html"
fig.write_html(savePath, auto_open=True)   


## Complete power trace collection

In [ ]:
from tqdm import tnrange, trange
import numpy as np
import time

ktp = cw.ktp.Basic()
ktp.fixed_key = False

traces = []
N = 10000  # Number of traces
prjDir = "/mnt/hgfs/trace_key_f051_10kAdd_061024/"

for fileNum in range(256):
    project = cw.create_project(prjDir + "key_" + str(fileNum) + ".cwp", overwrite=True)

    for i in trange(N, desc='Capturing traces:' + str(fileNum)):
        key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
        key[0] = fileNum

        trace = cw.capture_trace(scope, target, text, key)
        if trace is None:
            continue
        project.traces.append(trace)

    try:
        print(scope.adc.trig_count) # print if this exists
    except:
        pass
    project.save()
    project.close()

## Convert cw data to npz 

In [ ]:
## Use this to save to npz, pkl format is not compatible across different versions of python.
from tqdm import tnrange, trange
import chipwhisperer as cw
import pandas as pd
import os
import numpy as np

## dataFrame to store all the traces
df_new = pd.DataFrame(columns=['trace', 'key'])
traceList = []
keyList = []

# ##Adding traces for not complete 
# keysNotDone = [204, 228, 244, 245]
# count = 0
prjDir = "traces/f303_trial_081924_preCompiled.cwp"
saveDir = "/mnt/hgfs/trial_dir/"

df_new = pd.DataFrame(columns=['trace', 'key'])
traceList = []
keyList = []
new_project = cw.open_project(prjDir)
for index, trace in enumerate(new_project.traces):
    #df_new = df_new.append({'trace': trace.wave, 'key': trace.key}, ignore_index=True)
    traceList.append(trace.wave)
    keyList.append(trace.key)

##Create trace and key dataFrame        
df_new["trace"] = traceList
df_new["key"] = keyList

##Shuffle the dataframe before splitting in test, dev and train
df_new = df_new.sample(frac=1).reset_index(drop=True)


##Save it to pkl
saveTrain = saveDir + "/train_f303_preCompiled_gain30.npz"


if (not os.path.exists(saveTrain)):
    np.savez_compressed(saveTrain, data=df_new)
    print("Saved to %s" %(saveTrain))

In [ ]:
data_load = np.load(saveTrain, allow_pickle=True)["data"]
data_load.shape

In [ ]:
data_load[1]

## Create histogram plot to see what bounds the traces

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import chipwhisperer as cw
import plotly.graph_objects as go

fig = go.Figure()
for idx in [1,2]:
    traceList = None
    project = cw.open_project("/mnt/hgfs/cutoff_magnitude_traces/func" + str(idx) + "_trace.cwp")
    print(project.location)

    for i, trace in enumerate(project.traces[0:5000]):
        if(i==0):
            traceList = trace.wave.reshape(len(trace.wave),1)
        else:
            traceList = np.concatenate((traceList, trace.wave.reshape(len(trace.wave),1)), axis=1)
    fig.add_trace(go.Histogram(x=traceList.flatten(), nbinsx=100, name="Func_" + str(idx), histnorm="probability"))

##Add traces from f415 device
for idx in [0, 150, 255]:
    traceList = None
    project = cw.open_project("/mnt/hgfs/trace_key_f4_lat0_30db/key_" + str(idx) + ".cwp")
    print(project.location)

    for i, trace in enumerate(project.traces[:5000]):
        if(i==0):
            traceList = trace.wave.reshape(len(trace.wave),1)
        else:
            traceList = np.concatenate((traceList, trace.wave.reshape(len(trace.wave),1)), axis=1)

    fig.add_trace(go.Histogram(x=traceList.flatten(), nbinsx=100, name="key_" + str(idx), histnorm="probability"))
    
    
# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)

fig.update_layout(
    title='Histogram random function and raw traces from F415 taken from CW ',
    xaxis_title='Amplitude bins',
    yaxis_title='probability'
)

fig.add_vline(x=0.13, line_dash="dot",
              annotation_text="x= 0.13",
             annotation_position = "bottom left",
             annotation_font_size=20,
             )

fig.add_vline(x=-0.01, line_dash="dot",
              annotation_text="x= -0.01",
             annotation_position = "bottom left",
             annotation_font_size=20,
             )

savePath = "/mnt/hgfs/cutoff_magnitude_traces/cuttoff_histogram_with_f415.html"
fig.write_html(savePath, auto_open=True)



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import chipwhisperer as cw
import plotly.graph_objects as go

idx=0
project = cw.open_project("/mnt/hgfs/trace_key_f4_lat0_30db/key_" + str(idx) + ".cwp")
print(project.location)

y=project.traces[0].wave
xShape = project.traces[0].wave.shape[0]
x=np.linspace(0,xShape-1, xShape)

fig = go.Figure()

for key in [0, 129, 43, 85, 127, 154, 193, 210, 255]:
    project = cw.open_project("/mnt/hgfs/trace_key_f4_lat0_30db/key_" + str(key) + ".cwp")
    print(project.location)

    y=project.traces[0].wave
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='key_' + str(key)))

fig.update_layout(
    title='F415 raw power trace',
    xaxis_title='number of samples',
    yaxis_title='amplitude'
)
fig.add_hrect(y0=-0.01, y1=0.13, line_width=0, fillcolor="red", opacity=0.2)
fig.add_hline(y=-0.01, line_dash="dot",
              annotation_text="y= -0.01",
             annotation_position = "bottom left",
             annotation_font_size=20,
             )

fig.add_hline(y=0.13, line_dash="dot",
              annotation_text="y= 0.13",
             annotation_position = "top left",
             annotation_font_size=20,
             )

savePath = "/mnt/hgfs/cutoff_magnitude_traces/cuttoff_band_multiple_keys.html"
fig.write_html(savePath, auto_open=True)

#fig.show()

## Average spikes from nearby points

In [ ]:
scope.dis()
target.dis()